In [74]:
import json
import pandas as pd
import random
p = 0.2  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
raw_data_with_measures = pd.read_csv(
         '/content/data.csv', 
         skiprows=lambda i: i>0 and random.random() > p)

In [75]:
raw_data_with_measures['artists'] = raw_data_with_measures['artists'].apply(lambda x: x[1:-1].split(', ')) # makes the string lists in artists column into actual lists

raw_data_with_measures = raw_data_with_measures.explode('artists') # opens the lists so that each artist is on a song has his own row with the song

raw_data_with_measures['artists'] = raw_data_with_measures['artists'].apply(lambda x: x.strip("'")) #  we want to take off the quotes on the artists

raw_data =  raw_data_with_measures.loc[:,['artists','id','name','popularity','year']] # we will just focus on these columns for now, and rename our data to raw_data


raw_data

,artists,id,name,popularity,year
0,Felix Mendelssohn,6NxAf7M8DNHOBTmEd3JSO5,Scherzo a capriccio: Presto,0,1928
0,Vladimir Horowitz,6NxAf7M8DNHOBTmEd3JSO5,Scherzo a capriccio: Presto,0,1928
1,Ludwig van Beethoven,6Rwn56jcC0TdGQzbRl7NGw,"Symphony No. 5 in C Minor, Op. 67: 3. Allegro",0,1928
1,Staatskapelle Berlin,6Rwn56jcC0TdGQzbRl7NGw,"Symphony No. 5 in C Minor, Op. 67: 3. Allegro",0,1928
1,Richard Strauss,6Rwn56jcC0TdGQzbRl7NGw,"Symphony No. 5 in C Minor, Op. 67: 3. Allegro",0,1928
...,...,...,...,...,...
33881,YoungBoy Never Broke Again,0KVEBQA98JvkmB5ZydQ13k,Fine By Time,64,2020
33882,Trey Songz,5QZ11AHm7xiytOGXGlxQi5,Back Home (feat. Summer Walker),69,2020
33882,Summer Walker,5QZ11AHm7xiytOGXGlxQi5,Back Home (feat. Summer Walker),69,2020
33883,Leon Bridges,1ehhGlTvjtHo2e4xJFB0SZ,Sweeter (feat. Terrace Martin),64,2020


In [ ]:
raw_data_genre = pd.read_csv('data_w_genres.csv')
raw_data_genre.head(4)

In [ ]:
raw_data_genre['genres'] = raw_data_genre['genres'].apply(lambda x: x[1:-1].split(', '))  # makes it into actual list instead of string list

for i in raw_data_genre.index:

    raw_data_genre['genres'].loc[i] = list(filter(None, raw_data_genre['genres'][i]))  # filters out empty values in each list
    

raw_data_genre.head(10)

In [ ]:
missing_genre_artists = raw_data_genre[raw_data_genre['genres'].map(lambda x: len(x)) < 1]
missing_genre_artists

In [ ]:
missing_genre_artists_split = missing_genre_artists.copy()
missing_genre_artists_split['artists'] = missing_genre_artists_split['artists'].apply(lambda x: x.split())  # split into words
missing_genre_artists_split

In [ ]:
# lets find most common words in artists

def word_count(data):

    all_words = []  
    for elmnt in data['artists']:  
        all_words += elmnt  

    val_counts = pd.Series(all_words).value_counts()

    return val_counts.head(40)

word_count(missing_genre_artists_split)

In [ ]:
pd.options.mode.chained_assignment = None 

for i in raw_data_genre.index:
    
    if 'Broadway' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'broadway'")
    if 'Motion Picture' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'movie tunes'")
    if 'Cast' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'ensemble'")     # we will put any cast artists under ensemble genre 
    if 'Ensemble' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'ensemble'")  
    if 'Orchestra' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Orquesta' in raw_data_genre['artists'].loc[i]:              # these are just vara
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Orchestre' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Chorus' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'chorus'")
    if 'Choir' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'choir'")
        
    raw_data_genre['genres'].loc[i] = list(dict.fromkeys(raw_data_genre['genres'].loc[i])) # removes duplicates
    

raw_data_genre.head(12)

In [ ]:
#create final df that removes artists without genres 
final_data_genre = raw_data_genre[raw_data_genre['genres'].map(lambda x: len(x)) > 0] # removes empty lists
final_data_genre = final_data_genre.reset_index(drop=True)
final_data_genre = final_data_genre.drop('count', axis =1 )
final_data_genre.rename(columns={'popularity': 'Artist Popularity'}, inplace=True)
final_data_genre['Artist Popularity'] = final_data_genre['Artist Popularity'].astype(int)
final_data_genre

In [ ]:
raw_data_with_measures

In [83]:
merged_df = raw_data.merge(final_data_genre, how = 'inner', on = ['artists'])
merged_df_copy = merged_df.copy()
merged_df_copy.rename(columns={'artists': 'Artist', 'name':'Song Name','popularity':'Song Popularity','year':'Year','genres':'Genres'}, inplace=True)
merged_df_copy['Genres_cleaned'] = [','.join(map(str, l)) for l in merged_df_copy['Genres']]
merged_df_copy.drop(columns=['Genres'], inplace=True)
final_df = merged_df_copy.drop(columns=['key', 'mode'])


In [104]:
final_df.head(2)

,Artist,id,Song Name,Song Popularity,Year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,Artist Popularity,Genres_cleaned
0,Felix Mendelssohn,6NxAf7M8DNHOBTmEd3JSO5,Scherzo a capriccio: Presto,0,1928,0.95083,0.330667,334917.267857,0.144775,0.629957,0.145538,-20.846027,0.041148,102.961884,0.280798,4,"'classical','early romantic era'"
1,Felix Mendelssohn,4vBZRf6KWbMryfuuglDdCi,"Incidental Music to A Midsummer Night's Dream,...",0,1942,0.95083,0.330667,334917.267857,0.144775,0.629957,0.145538,-20.846027,0.041148,102.961884,0.280798,4,"'classical','early romantic era'"


In [100]:
#dummy encode genres 
new_df = pd.concat([final_df.drop('Genres_cleaned', 1), final_df['Genres_cleaned'].str.get_dummies(sep=",")], 1)


In [105]:
from sklearn.preprocessing import LabelEncoder
artist_encoder = LabelEncoder()
song_encoder = LabelEncoder()
new_df['Artist'] = artist_encoder.fit_transform(new_df['Artist'])
new_df['Song Name'] = song_encoder.fit_transform(new_df['Song Name'])

In [106]:
new_df.shape

(38815, 2059)

In [141]:
from sklearn.neighbors import NearestNeighbors

# Instantiate and fit knn to the correct columns
knn = NearestNeighbors(n_neighbors=1000)
knn.fit(new_df[new_df.columns[5:]])

def predict(track_id):

    obs = new_df.index[new_df['id'] == track_id]
    series = new_df.iloc[obs, 5:].to_numpy()

    neighbors = knn.kneighbors(series)
    new_obs = neighbors[1][0][6:20]
    return list(new_df.loc[new_obs, 'id'])

In [142]:
track_id = '6NxAf7M8DNHOBTmEd3JSO5'

obs = new_df.index[new_df['id'] == track_id]
obs

Int64Index([0, 24], dtype='int64')

In [143]:
predict('6NxAf7M8DNHOBTmEd3JSO5')

['6n9eKNXmhq7cRyr1DduJhF',
 '26GqCNd8YIYM7P5lHq4waV',
 '4VKbuRUzNREMZCX8FyrSkj',
 '2thpB5Uz5y3dvf4RJBRzOZ',
 '1KpZV5L3I2PWFi1zIXFLkI',
 '0S5jeGtL75XWsWgOmzP4Ji',
 '0r0rosy938OmJnh6v4EOyP',
 '2BxDpMy1E6Ba4hu8E2TY5F',
 '2cUF3mpuIHAwicXOVo6F8L',
 '0MkL6nGFtee1YEfvwK1C0C',
 '4z5kA10HBpAVz59uArXXZa',
 '3EE8z3wY7NFgjmBbW7tItP',
 '7IcPmAPkibXiIbmRvzrKUb',
 '3eqsh8q9i9KECm6p93fT4h']

In [138]:
final_df.loc[27080]

Artist                                                      Kurt Masur
id                                              1KpZV5L3I2PWFi1zIXFLkI
Song Name            Mendelssohn: Violin Concerto in E Minor Op. 64...
Song Popularity                                                     37
Year                                                              1993
acousticness                                                  0.975667
danceability                                                    0.1724
duration_ms                                                     521855
energy                                                        0.130911
instrumentalness                                              0.441744
liveness                                                      0.153267
loudness                                                      -20.7314
speechiness                                                  0.0423444
tempo                                                            98.05
valenc

In [136]:
final_df.loc[131]

Artist                                               Vladimir Horowitz
id                                              4VKbuRUzNREMZCX8FyrSkj
Song Name                                           May Breezes, No. 1
Song Popularity                                                      0
Year                                                              1928
acousticness                                                   0.99007
danceability                                                   0.34321
duration_ms                                                     266541
energy                                                        0.118844
instrumentalness                                              0.879508
liveness                                                      0.183812
loudness                                                      -23.1934
speechiness                                                  0.0433598
tempo                                                          94.9007
valenc

In [117]:
input = new_df[new_df['id'] == '1KpZV5L3I2PWFi1zIXFLkI']

In [118]:
print(input)

    Artist                      id  ...  'zolo'  'zydeco'
0     2624  6NxAf7M8DNHOBTmEd3JSO5  ...       0         0
24    8837  6NxAf7M8DNHOBTmEd3JSO5  ...       0         0

[2 rows x 2059 columns]


In [137]:
output = new_df[new_df['id'] == '1KpZV5L3I2PWFi1zIXFLkI']
print(output)

       Artist                      id  ...  'zolo'  'zydeco'
9        2624  1KpZV5L3I2PWFi1zIXFLkI  ...       0         0
27076    2993  1KpZV5L3I2PWFi1zIXFLkI  ...       0         0
27080    4469  1KpZV5L3I2PWFi1zIXFLkI  ...       0         0
31761    5348  1KpZV5L3I2PWFi1zIXFLkI  ...       0         0

[4 rows x 2059 columns]


In [125]:
new_df.loc[0]

Artist                               2624
id                 6NxAf7M8DNHOBTmEd3JSO5
Song Name                           20075
Song Popularity                         0
Year                                 1928
                            ...          
'yugoslav rock'                         0
'zapstep'                               0
'zhongguo feng'                         0
'zolo'                                  0
'zydeco'                                0
Name: 0, Length: 2059, dtype: object

In [150]:
import joblib

joblib.dump(knn, "KNN.joblib", compress=2, protocol=None, cache_size=None)


['KNN.joblib']

In [151]:
!ls -lh

total 43M
-rw-r--r-- 1 root root  29M Aug 25 18:59 data.csv
-rw-r--r-- 1 root root 5.1M Aug 25 18:58 data_w_genres.csv
-rw-r--r-- 1 root root 9.1M Aug 25 21:50 KNN.joblib
drwxr-xr-x 1 root root 4.0K Aug 24 16:35 sample_data


In [152]:
#load model from pckle joblib file 
knn = joblib.load('KNN.joblib')

def predict(track_id):

    obs = new_df.index[new_df['id'] == track_id]
    series = new_df.iloc[obs, 5:].to_numpy()

    neighbors = knn.kneighbors(series)
    new_obs = neighbors[1][0][6:20]
    return list(new_df.loc[new_obs, 'id'])

In [153]:
predict('6NxAf7M8DNHOBTmEd3JSO5')

['6n9eKNXmhq7cRyr1DduJhF',
 '26GqCNd8YIYM7P5lHq4waV',
 '4VKbuRUzNREMZCX8FyrSkj',
 '2thpB5Uz5y3dvf4RJBRzOZ',
 '1KpZV5L3I2PWFi1zIXFLkI',
 '0S5jeGtL75XWsWgOmzP4Ji',
 '0r0rosy938OmJnh6v4EOyP',
 '2BxDpMy1E6Ba4hu8E2TY5F',
 '2cUF3mpuIHAwicXOVo6F8L',
 '0MkL6nGFtee1YEfvwK1C0C',
 '4z5kA10HBpAVz59uArXXZa',
 '3EE8z3wY7NFgjmBbW7tItP',
 '7IcPmAPkibXiIbmRvzrKUb',
 '3eqsh8q9i9KECm6p93fT4h']